#1. 데이터 읽기

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['class'] = cancer.target
df.head()

#2. 모델 만들기
> voting_classifier(분류기 리스트, voting="hard, soft")

##2.1 데이터 분류

In [ ]:
df_data = df.iloc[:, :-1]
df_labels = df.iloc[:,-1]

> df 생성 시 마지막 위치에 class feature를 추가 하였음으로 마지막 이전 데이터와 마지막 데이터만 추출하여 data와 labels로 사용할 수 있다. -1은 index의 마지막 위치를 의미함으로 df_data는 마지막 전까지의 데이터를 얻어오게 되는 것이며 df_labels는 마지막 값만 얻어 오게 되는 것이다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=62)

##2.2 모델 만들기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression(max_iter=5000)
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

> 로지스틱 회귀와 KNN 두 가지 알고리즘을 이용하여 보팅을 진행할 것이다. VotingClassifier의 첫 인자는 알고리즘 리스트가 들어 간다. 현재 ('lr_id', lr) 이와 같은 형식으로 들어 갔는데 이 부분은 추후 하이퍼 파라미터 적용 시 로지스틱 회귀와 KNN에서 사용되는 파라미터가 다르기 때문에 이를 식별하는 식별자로 lr_id을 만들어 둔 것이다.

##2.3 학습 및 예측

In [ ]:
from sklearn.metrics import accuracy_score

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

accuracy_score(y_test, pred)

0.9181286549707602

##2.4 모델별 성능 평가

In [ ]:
for clf in [lr, knn]:
  # print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 : ', accuracy_score(y_test, pred))

LogisticRegression 의 정확도 :  0.9181286549707602
KNeighborsClassifier 의 정확도 :  0.8888888888888888


#3. 스케일링을 통한 모델 성능 높이기
> 로지스틱 회귀의 경우 데이터를 스케일링하게 되면 성능이 향샹됨으로 StanardScaler를 이용할 것이다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=62)

lr = LogisticRegression(max_iter=5000)
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred))

for clf in [lr, knn]:
  # print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 : ', accuracy_score(y_test, pred))

print('========================================================')

# 표준화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)


vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred))

for clf in [lr, knn]:
  # print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 : ', accuracy_score(y_test, pred))

voting 정확도 :  0.9181286549707602
LogisticRegression 의 정확도 :  0.9181286549707602
KNeighborsClassifier 의 정확도 :  0.8888888888888888
voting 정확도 :  0.9473684210526315
LogisticRegression 의 정확도 :  0.9590643274853801
KNeighborsClassifier 의 정확도 :  0.935672514619883


#4. 교차 검증

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

for clf in [vo, lr, knn]:
  scores = cross_val_score(clf, np.concatenate((X_train, X_test)), np.concatenate((y_train, y_test)), scoring='accuracy', cv=5)

  print(clf.__class__.__name__)
  print(scores)
  print(np.mean(scores))
  print()

> np.concatenate은 두 데이터를 합하는 함수이다.

#5. 하이퍼 파라미터

In [ ]:
from sklearn.model_selection import GridSearchCV

def get_best_hyper_paramerter():
  params =  {
      # voting 생성 시 사용한 이름 + '__' + 파라미터
    'lr_id__C' : logreg_list,
    'knn_id__n_neighbors' : n_neighbors_list,
    'knn_id__weights' : ["uniform", "distance"],
    'knn_id__metric' : ['euclidean', 'manhattan', 'minkowski']
  }

  grid_cv = GridSearchCV(vo, param_grid=params, scoring='accuracy', cv=5)
  grid_cv.fit(X_train, y_train)
  print('Best Score: ', grid_cv.best_score_)
  print('Best Hyper Parameter: ', grid_cv.best_params_)

In [ ]:
logreg_list = [0.001, 0.01 ,0.1 ,1 ,10 ,25 ,50 ,100]
n_neighbors_list = range(1, 100, 10)

get_best_hyper_paramerter()

Best Score:  0.9824050632911392
Best Hyper Parameter:  {'knn_id__metric': 'euclidean', 'knn_id__n_neighbors': 11, 'knn_id__weights': 'distance', 'lr_id__C': 10}


In [ ]:
# 2, 6, 10 ~~~
logreg_list = range(2, 26, 4)
# 1, 6, 11
n_neighbors_list = range(1, 21, 5)

get_best_hyper_paramerter()

Best Score:  0.9849050632911392
Best Hyper Parameter:  {'knn_id__metric': 'euclidean', 'knn_id__n_neighbors': 6, 'knn_id__weights': 'uniform', 'lr_id__C': 14}


In [ ]:
# 2, 6, 10 ~~~
logreg_list = range(11, 17, 1)
# 1, 6, 11
n_neighbors_list = range(2, 10, 1)

get_best_hyper_paramerter()

Best Score:  0.9849367088607595
Best Hyper Parameter:  {'knn_id__metric': 'manhattan', 'knn_id__n_neighbors': 3, 'knn_id__weights': 'uniform', 'lr_id__C': 14}


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=62)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

lr = LogisticRegression(C=14, max_iter=5000)
knn = KNeighborsClassifier(metric= 'manhattan', n_neighbors=3, weights= 'uniform')
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred))

for clf in [lr, knn]:
  # print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 : ', accuracy_score(y_test, pred))

voting 정확도 :  0.9532163742690059
LogisticRegression 의 정확도 :  0.9649122807017544
KNeighborsClassifier 의 정확도 :  0.9239766081871345
